<a href="https://colab.research.google.com/github/EtzionR/LM4GeoAI/blob/main/Geo_LLM_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex 3: Geo LLM Agents

### created by Etzion Harari | Geo-AI Course

[**https://github.com/EtzionR/LM4GeoAI**](https://github.com/EtzionR/LM4GeoAI)

In [1]:
!pip install cohere
!pip install osmnx
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.1 MB/s eta 0:00:00


In [2]:
from difflib import get_close_matches
from getpass import getpass

import numpy as np
import osmnx as ox
import folium
import cohere

Cohere models: [https://docs.cohere.com/docs/models#command](https://docs.cohere.com/docs/models#command)

Cohere MCP: [https://docs.cohere.com/page/basic-tool-use](https://docs.cohere.com/page/basic-tool-use)

In [3]:
MODEL  = "command-a-03-2025"
TEMP = .01

Get your Cohere API Key:

[https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys)

In [4]:
KEY = getpass("Please Enter COHERE KEY:\n")

len(KEY)

Please Enter COHERE KEY:
··········


40

In [5]:
client = cohere.Client(api_key=KEY)
client

In [6]:
QUERY = 'What is panda bear?'

output = client.chat(model=MODEL,
                     message=QUERY,
                     temperature=TEMP).text

print(output)

A panda bear, commonly referred to as a giant panda, is a species of bear native to central China. Scientifically known as *Ailuropoda melanoleuca*, the giant panda is easily recognizable by its distinctive black and white coat. Here are some key characteristics and facts about panda bears:

1. **Appearance**: Pandas have a white body with black patches around their eyes, ears, and across their shoulders and legs. They have a robust build, with a stocky body and a short tail.

2. **Diet**: Despite being classified as carnivores, pandas primarily eat bamboo, which makes up about 99% of their diet. They also occasionally consume other vegetation, small animals, and fish.

3. **Habitat**: Pandas are found in the temperate forests of the mountain ranges in central China, particularly in Sichuan, Shaanxi, and Gansu provinces. They prefer areas with dense bamboo growth.

4. **Behavior**: Pandas are generally solitary animals, except during the breeding season. They are excellent climbers and

In [7]:
def get_pois_near_adress(location_adress, place_type, radius=500):
    """
    Load OSM POIs from given place_type

    :location_adress: location adress for searching OSM
    :place_type: type of point of interest (POI)
    :radius: radius around location_adress (in meters, int, default: 500)

    :return: selected dataframe
    """

    # Load data from OSM
    out = ox.features_from_address(location_adress, {"amenity": True}, radius)
    out = out[['name','amenity','geometry']]
    out = out.rename(columns={'amenity':'place_type'})

    # Find the closets place_type (using SequenceMatcher)
    place_type = get_close_matches(place_type,
                                   [*out.place_type.unique()],
                                   n=1,
                                   cutoff=.0)[0]

    # select the place_type
    dataframe = out[out.place_type==place_type]

    return dataframe

# ------------------------------------------------------------------------------


# get_place_info application:

location_adress = 'מגדלי עזריאלי'
place_type      = 'fual'           # not fuel!

get_pois_near_adress(location_adress, place_type)

name place_type                   geometry
element id                                                  
node    442893138  דלק       fuel  POINT (34.79586 32.07468)
        442893144  דלק       fuel  POINT (34.79449 32.07144)
        442898671   פז       fuel   POINT (34.79724 32.0719)

In [8]:
def shortest_path_between_adresses(adress_1, adress_2,  gap=.25):
    """
    Calculate shortest path between two adresses using OSM roads network

    :adress_1: origin adress (source)
    :adress_2: destination adress (target)
    :gap: buffer ratio from bbox (float, default: .25)

    :return: shortest path between adress_1 to adress_2 (coordinates)
    """

    # gecode to both adresses
    y1, x1 = ox.geocode(adress_1)
    y2, x2 = ox.geocode(adress_2)

    # build the bbox
    delta = max(abs(x1-x2)*gap, abs(y1-y2)*gap)
    bbox = [min(x1,x2)-delta, min(y1,y2)-delta, max(x1,x2)+delta, max(y1,y2)+delta]

    # get the roads network graph
    G = ox.graph.graph_from_bbox(bbox, truncate_by_edge =True, network_type ='drive', simplify =False)

    # find the origin & destination nodes
    node1 = ox.distance.nearest_nodes(G, x1, y1)
    node2 = ox.distance.nearest_nodes(G, x2, y2)

    # calculate path between origin to destination
    route = ox.routing.shortest_path(G, orig=node1, dest=node2)
    route_xy = [(G.nodes[osmid]['x'], G.nodes[osmid]['y']) for osmid in route]

    return route_xy

# ------------------------------------------------------------------------------


# shortest_path_between_adresses application:

ad1 = "אוניברסיטת תל אביב"
ad2 = 'מגדלור רידינג'

path = shortest_path_between_adresses(ad1, ad2)

print(f'Number of nodes in the output path: {len(path)}\n\nFirst 5 points:\n{path[:5]}')

Number of nodes in the output path: 128

First 5 points:
[(34.8037004, 32.1107766), (34.8037061, 32.1107118), (34.8037047, 32.1105204), (34.8036907, 32.1103309), (34.8036844, 32.1101144)]


In [9]:
system_prompt = """You are an AI assistant with access to the following tools:
get_pois_near_adress tool, which enable you to run osm query.
shortest_path_between_adresses tool, which enable you to calculate route between to given place names."""

tools = [
    {
        "name": "get_pois_near_adress",
        "description": "can return every type of geographic location near some given adress",
        "parameter_definitions": {
            "location_adress": {
                "description": "free text location adress",
                "type": "str",
                "required": True
            },
            "place_type": {
                "description": "type of point of interest (POI)",
                "type": "str",
                "required": True
            },
            "radius": {
                "description": "radius of search in meters",
                "type": "float",
                "required": False
            }
        }
    },

    {
        "name": "shortest_path_between_adresses",
        "description": "Calculate shortest path between two adresses using OSM roads network",
        "parameter_definitions": {
            "adress_1": {
                "description": "adress of the origin point (free text)",
                "type": "str",
                "required": True
            },
            "adress_2": {
                "description": "adress of the destination point (free text)",
                "type": "str",
                "required": True
            },
            "gap": {
                "description": "buffer ratio of the bbox to load roads network",
                "type": "float",
                "required": False
            }
        }
    },
]

function_maps = {get_pois_near_adress.__name__: get_pois_near_adress,
                 shortest_path_between_adresses.__name__: shortest_path_between_adresses}
function_maps

{'get_pois_near_adress': <function __main__.get_pois_near_adress(location_adress, place_type, radius=500)>,
 'shortest_path_between_adresses': <function __main__.shortest_path_between_adresses(adress_1, adress_2, gap=0.25)>}

In [10]:
HEB = 'מגדלי עזריאלי'

user_prompt = f"I want you to return every cafe near {HEB}"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = function_maps[response.tool_calls[0].name](**response.tool_calls[0].parameters)
out

Model output: I will use the get_pois_near_adress tool to find all the cafes near מגדלי עזריאלי.




name place_type                   geometry
element id                                                            
node    553022802       קפה שרגא       cafe    POINT (34.7903 32.0707)
        1207157419           Uno       cafe  POINT (34.78908 32.07748)
        3465156312        רולדין       cafe  POINT (34.78693 32.07173)
        6469411087        לנדוור       cafe  POINT (34.79164 32.07452)
        6469411088       קפה קפה       cafe   POINT (34.79172 32.0747)
        6644165508         Aroma       cafe  POINT (34.78769 32.07308)
        7211406506   Max Brenner       cafe   POINT (34.78677 32.0715)
        10784964980      Arcaffe       cafe  POINT (34.78864 32.07829)
        10812416453       BEITEA       cafe  POINT (34.78798 32.07223)
        11802167414      Arcaffe       cafe  POINT (34.79304 32.07731)
        12032015683          mae       cafe  POINT (34.78829 32.07195)
        12139988888     קפה עוטף       cafe  POINT (34.78739 32.07308)
        12635329119       רולדין       cafe   POINT (34.79232 32.0743)
        13338979218         Biga       cafe  POINT (34.78691 32.07308)

In [11]:
fmap = folium.Map(location=[out.geometry.y.median(), out.geometry.x.median()],
                  zoom_start=15.5, width=500, height=500)

folium.GeoJson(out).add_to(fmap)

fmap

In [12]:
user_prompt = f"אני רוצה לדעת מה המסלול מחוף מציצים למגדלי עזריאלי"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = function_maps[response.tool_calls[0].name](**response.tool_calls[0].parameters)

print(f'Output length: {len(out)}')

Model output: אשתמש ב-shortest_path_between_adresses tool כדי למצוא את המסלול הקצר ביותר מחוף מציצים למגדלי עזריאלי.


Output length: 137


In [14]:
import numpy as np

fmap = folium.Map(location=[np.array(out)[:,1].mean(), np.array(out)[:,0].mean()],
                  zoom_start=14, width=500, height=500)

folium.PolyLine(
    locations=[(y,x) for x,y in out],
    color="blue",
    weight=5,
    tooltip=user_prompt,
).add_to(fmap)

fmap

# -----------------------------------------------------------------------

In [16]:

import wikipedia

# search by query (not exact match)
results = wikipedia.search("machine learning in healthcare")
print(results)

# pick the first match
page = wikipedia.page(results[0])
print(page.title)
print(page.content)

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=a3d86fe87dfd5ed466e5a6245b7d0a25ce2faf8d0b9a46dad71d26fa3e5f855c
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
['Artificial intelligence in healthcare', 'Machine learning', 'Health informatics', 'Explainable artificial intelligence', 'Deep reinforcement learning', 'Suchi Saria', 'Deep learning', 'Political methodology', 'Edward Y. Chang', 'Artificial intelligence']
Artificial intelligence in healthcare
Artificial intelligence in healthcare is the application of artificial intelligence (AI) to analyze and understand complex medical and healthcare data. In some cases, it can exceed or augment human capabilities by providing better or faster ways to diagnose, treat, or prevent disease.
As the widespread use of artificial intelligence in healthcare is still relatively new

In [22]:
point = (40.75805, -73.985716)

options = ox.features.features_from_point(center_point = point, dist=100, tags = {'amenity':True})[['geometry','name']].dropna().drop_duplicates()

options

geometry  \
element id                                                               
node    368051619                            POINT (-73.98462 40.7589)   
        763650163                           POINT (-73.98663 40.75822)   
        1740694091                          POINT (-73.98472 40.75878)   
        1752822408                          POINT (-73.98621 40.75806)   
        2678466841                          POINT (-73.98644 40.75765)   
        2678466842                          POINT (-73.98639 40.75723)   
        2717293824                          POINT (-73.98671 40.75751)   
        2782000317                          POINT (-73.98655 40.75744)   
        4385240481                          POINT (-73.98642 40.75874)   
        5172045822                          POINT (-73.98684 40.75756)   
        10960870136                         POINT (-73.98533 40.75773)   
        12175585351                          POINT (-73.98618 40.7589)   
        12237980160                         POINT (-73.98602 40.75834)   
        12244426994                         POINT (-73.98587 40.75851)   
        12246616257                         POINT (-73.98656 40.75856)   
        12488728937                         POINT (-73.98634 40.75731)   
        12488728940                         POINT (-73.98539 40.75767)   
        12917522701                         POINT (-73.98469 40.75744)   
        12968777478                          POINT (-73.98483 40.7575)   
way     266149299    POLYGON ((-73.98735 40.75834, -73.98721 40.758...   
        266149330    POLYGON ((-73.98665 40.75884, -73.98689 40.758...   
        266170751    POLYGON ((-73.98435 40.75705, -73.98438 40.757...   
        266170755    POLYGON ((-73.98442 40.75789, -73.9843 40.7578...   

                                             name  
element id                                         
node    368051619                  Palace Theater  
        763650163                        Junior's  
        1740694091                     McDonald's  
        1752822408               Minskoff Theatre  
        2678466841            PlayStation Theater  
        2678466842      Bubba Gump Shrimp Company  
        2717293824                      Carmine's  
        2782000317                  Ben & Jerry's  
        4385240481   The View Restaurant & Lounge  
        5172045822   Guy’s American Kitchen & Bar  
        10960870136                    McDonald's  
        12175585351               Marquis Theatre  
        12237980160                  Anita Gelato  
        12244426994                   Revel & Rye  
        12246616257                     Starbucks  
        12488728937                Raising Cane's  
        12488728940                       Popeyes  
        12917522701                     Starbucks  
        12968777478                   Duane Reade  
way     266149299                   Booth Theatre  
        266149330         Richard Rodgers Theatre  
        266170751                  Hudson Theatre  
        266170755                  Lyceum Theatre

In [26]:


WGS84_UTM_18N = "EPSG:32618"

options.to_crs(WGS84_UTM_18N)

geometry  \
element id                                                               
node    368051619                       POINT (585706.386 4512489.115)   
        763650163                       POINT (585537.594 4512411.092)   
        1740694091                      POINT (585697.819 4512475.139)   
        1752822408                      POINT (585573.359 4512394.618)   
        2678466841                      POINT (585553.898 4512347.997)   
        2678466842                      POINT (585558.851 4512302.169)   
        2717293824                      POINT (585531.246 4512332.826)   
        2782000317                      POINT (585545.075 4512324.692)   
        4385240481                       POINT (585554.785 4512470.01)   
        5172045822                      POINT (585520.716 4512338.155)   
        10960870136                      POINT (585647.58 4512359.028)   
        12175585351                      POINT (585574.01 4512488.029)   
        12237980160                     POINT (585588.434 4512425.557)   
        12244426994                      POINT (585601.32 4512444.269)   
        12246616257                     POINT (585542.515 4512449.129)   
        12488728937                     POINT (585562.576 4512310.739)   
        12488728940                     POINT (585642.778 4512351.645)   
        12917522701                     POINT (585702.273 4512327.275)   
        12968777478                     POINT (585690.055 4512333.329)   
way     266149299    POLYGON ((585476.675 4512424.289, 585487.915 4...   
        266149330    POLYGON ((585535.03 4512480.285, 585514.056 45...   
        266170751    POLYGON ((585731.408 4512283.492, 585728.366 4...   
        266170755    POLYGON ((585724.639 4512377.427, 585734.194 4...   

                                             name  
element id                                         
node    368051619                  Palace Theater  
        763650163                        Junior's  
        1740694091                     McDonald's  
        1752822408               Minskoff Theatre  
        2678466841            PlayStation Theater  
        2678466842      Bubba Gump Shrimp Company  
        2717293824                      Carmine's  
        2782000317                  Ben & Jerry's  
        4385240481   The View Restaurant & Lounge  
        5172045822   Guy’s American Kitchen & Bar  
        10960870136                    McDonald's  
        12175585351               Marquis Theatre  
        12237980160                  Anita Gelato  
        12244426994                   Revel & Rye  
        12246616257                     Starbucks  
        12488728937                Raising Cane's  
        12488728940                       Popeyes  
        12917522701                     Starbucks  
        12968777478                   Duane Reade  
way     266149299                   Booth Theatre  
        266149330         Richard Rodgers Theatre  
        266170751                  Hudson Theatre  
        266170755                  Lyceum Theatre

In [24]:
options

geometry  \
element id                                                               
node    368051619                            POINT (-73.98462 40.7589)   
        763650163                           POINT (-73.98663 40.75822)   
        1740694091                          POINT (-73.98472 40.75878)   
        1752822408                          POINT (-73.98621 40.75806)   
        2678466841                          POINT (-73.98644 40.75765)   
        2678466842                          POINT (-73.98639 40.75723)   
        2717293824                          POINT (-73.98671 40.75751)   
        2782000317                          POINT (-73.98655 40.75744)   
        4385240481                          POINT (-73.98642 40.75874)   
        5172045822                          POINT (-73.98684 40.75756)   
        10960870136                         POINT (-73.98533 40.75773)   
        12175585351                          POINT (-73.98618 40.7589)   
        12237980160                         POINT (-73.98602 40.75834)   
        12244426994                         POINT (-73.98587 40.75851)   
        12246616257                         POINT (-73.98656 40.75856)   
        12488728937                         POINT (-73.98634 40.75731)   
        12488728940                         POINT (-73.98539 40.75767)   
        12917522701                         POINT (-73.98469 40.75744)   
        12968777478                          POINT (-73.98483 40.7575)   
way     266149299    POLYGON ((-73.98735 40.75834, -73.98721 40.758...   
        266149330    POLYGON ((-73.98665 40.75884, -73.98689 40.758...   
        266170751    POLYGON ((-73.98435 40.75705, -73.98438 40.757...   
        266170755    POLYGON ((-73.98442 40.75789, -73.9843 40.7578...   

                                             name  
element id                                         
node    368051619                  Palace Theater  
        763650163                        Junior's  
        1740694091                     McDonald's  
        1752822408               Minskoff Theatre  
        2678466841            PlayStation Theater  
        2678466842      Bubba Gump Shrimp Company  
        2717293824                      Carmine's  
        2782000317                  Ben & Jerry's  
        4385240481   The View Restaurant & Lounge  
        5172045822   Guy’s American Kitchen & Bar  
        10960870136                    McDonald's  
        12175585351               Marquis Theatre  
        12237980160                  Anita Gelato  
        12244426994                   Revel & Rye  
        12246616257                     Starbucks  
        12488728937                Raising Cane's  
        12488728940                       Popeyes  
        12917522701                     Starbucks  
        12968777478                   Duane Reade  
way     266149299                   Booth Theatre  
        266149330         Richard Rodgers Theatre  
        266170751                  Hudson Theatre  
        266170755                  Lyceum Theatre

12